In [12]:
import pandas as pd
import numpy as np
import sys

In [13]:
uniqueid = 'uid_s'

In [14]:
from multiprocessing import Pool
def reader(n_subarray):
    return pd.read_csv('/disk1/hrb/python/data/surveys/ztf/calibStars/lc_{}.csv'.format(n_subarray))# usecols = [0,1,2,3,4,5], index_col = 0, dtype = {'mag': np.float32, 'magerr': np.float32, 'mjd': np.float64, 'uid': np.uint32})

def read_ztf():
    pool = Pool(4)
    df_list = pool.map(reader, [0,1,2,3])
    return pd.concat(df_list, ignore_index=True)

df_ztf = read_ztf()
df_ztf['filtercode'] = df_ztf['filtercode'].str[-1]

In [15]:
df_ztf

,uid_s,oid,mjd,mag,magerr,filtercode,magzp,clrcoeff,clrcounc,airmass,dist_x,ngoodobsrel
0,5,439209100007029,58256.459965,21.039665,0.168187,r,26.184282,0.080792,0.000004,1.326,0.342585,6
1,5,439209100007029,58286.462396,21.107273,0.172688,r,26.199413,0.087795,0.000004,1.257,0.342585,6
2,5,439209100007029,58329.334988,20.341053,0.119316,r,26.124464,0.071770,0.000006,1.253,0.342585,6
3,5,439209100007029,58342.322014,20.799389,0.152189,r,26.076647,0.083779,0.000004,1.291,0.342585,6
4,5,439209100007029,58343.315961,21.294666,0.185165,r,26.146166,0.086070,0.000006,1.284,0.342585,6
...,...,...,...,...,...,...,...,...,...,...,...,...
63537655,1006830,404214400002163,58347.453947,21.370272,0.229004,r,26.049042,0.097334,0.000013,1.827,0.143774,5
63537656,1006830,404214400002163,58345.497986,20.887083,0.191271,r,26.107546,0.089372,0.000011,1.485,0.143774,5
63537657,1006830,404214400002163,58345.483426,21.027519,0.202238,r,26.100943,0.086438,0.000013,1.586,0.143774,5
63537658,1006830,404214400002163,58368.442882,20.977371,0.198322,r,26.181648,0.102868,0.000011,1.419,0.143774,5


In [18]:
df_ztf = df_ztf.set_index('uid_s')
len(df_ztf.index.unique())

524749

In [21]:
np.savetxt('uids.txt',df_ztf.index.unique(), fmt='%i')

In [6]:
# Need to remove >1 arcsec otherwise we get neighbouring objects
df_ztf_meta = pd.read_csv('/disk1/hrb/python/data/surveys/ztf/calibStars/calibStars_ztf.txt', delimiter = '\s+', skiprows = 62, usecols=[1,3,6,12], names=['dist_x',uniqueid,'oid','ngoodobsrel'], index_col=uniqueid)
mask = df_ztf_meta['oid'].duplicated(keep='first')
df_ztf_meta = df_ztf_meta[~mask]
# df_ztf_meta = pd.read_csv('/disk1/hrb/python/data/surveys/ztf/meta_data/ztf_meta.txt', delimiter = '\s+', skiprows = 62, usecols=[1,5,7,15], names=['dist_x',uniqueid,'oid','ngoodobsrel'], index_col=2)
df_ztf_meta = df_ztf_meta[(df_ztf_meta['dist_x'] < 1) & (df_ztf_meta['ngoodobsrel'] > 0)]

/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
oids = pd.read_csv('/disk1/hrb/python/data/surveys/ztf/calibStars/calibStars_oids.txt', names=['oid'])

In [8]:
assert df_ztf_meta['oid'].is_unique, 'oid is not unique'

In [9]:
df_ztf_2 = df_ztf.merge(df_ztf_meta.reset_index(), on = 'oid', how='inner').set_index(uniqueid).sort_values(uniqueid)

In [10]:
df_ztf_2

,oid,mjd,mag,magerr,filtercode,magzp,clrcoeff,clrcounc,airmass,dist_x,ngoodobsrel
uid_s,,,,,,,,,,,
5,439209100007029,58256.459965,21.039665,0.168187,r,26.184282,0.080792,0.000004,1.326,0.342585,6
5,439209100007029,58286.462396,21.107273,0.172688,r,26.199413,0.087795,0.000004,1.257,0.342585,6
5,439209100007029,58329.334988,20.341053,0.119316,r,26.124464,0.071770,0.000006,1.253,0.342585,6
5,439209100007029,58342.322014,20.799389,0.152189,r,26.076647,0.083779,0.000004,1.291,0.342585,6
5,439209100007029,58343.315961,21.294666,0.185165,r,26.146166,0.086070,0.000006,1.284,0.342585,6
...,...,...,...,...,...,...,...,...,...,...,...
1006830,404214400002163,58347.453947,21.370272,0.229004,r,26.049042,0.097334,0.000013,1.827,0.143774,5
1006830,404214400002163,58345.497986,20.887083,0.191271,r,26.107546,0.089372,0.000011,1.485,0.143774,5
1006830,404214400002163,58345.483426,21.027519,0.202238,r,26.100943,0.086438,0.000013,1.586,0.143774,5


In [11]:
for i, df_chunk in enumerate(np.array_split(df_ztf_2,4)):
    df_chunk.to_csv('/disk1/hrb/python/data/surveys/ztf/calibStars/lc_{}.csv'.format(i))

In [ ]:
python/data/surveys/ztf/qsos/meta_data/ztf_meta.txt